In [1]:
import pandas as pd
import json
import requests

In [2]:
def remove_duplicate_rows(df, column_name_to_filter):
    grouped = df.groupby(column_name_to_filter)
    index = [gp_keys[0] for gp_keys in grouped.groups.values()]
    return df.reindex(index)

## Formatting output from "Session Buddy"

In [40]:
with open('data.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data["tabs"])

In [43]:
output_df = df[['title','url']]
output_df.insert( 2,'user', "dataradar")
output_df.insert( 3,'tags',"")
output_df.insert( 4,'twitter',True)
output_df.insert( 5,'auto_format',False)

output_df = remove_duplicate_rows(output_df, 'url')

In [44]:
output_df.to_json("output.json", orient="records", force_ascii=False)

## Check Uniqueness in consolidated.json File

In [132]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'url')

In [133]:
api_route = "https://www.feedcrunch.io/api/1.0/authenticated/get/article/exists/"

apikey_local = "91ce86bd-ae46-4250-b599-727c1970b609"
apikey_fc    = "4377190f-24a5-4ef7-a761-a877bf6218e3"

api_url = api_route + apikey_fc + "/"

def check_link_exists(url, user):
    r = requests.get(api_url+'?link='+url+'&posting_user='+user)
    try:
        return r.json()["exists"]
    except:
        return False

In [134]:
lines_2_delete = []

for index in range(df.shape[0]):
    
    url = df.iloc[index]['url']
    user = df.iloc[index]['user']
    
    if check_link_exists(url, user):
        lines_2_delete.append(index)
        print 'Duplicate Found: ' + url + " && user: " + user

In [135]:
df.drop(df.index[lines_2_delete], inplace=True)

df = df.sort_values(by=['url', 'title'], ascending=[True, True])

In [138]:
df.to_json("consolidated.json", orient="records", force_ascii=False)

## Describe Data

In [45]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

print "There are " + str(df.shape[0]) + " articles in dataset"
df = remove_duplicate_rows(df, 'url')
print "There are " + str(df.shape[0]) + " articles in dataset without duplicates"

There are 518 articles in dataset
There are 518 articles in dataset without duplicates


## Sorting Data

In [37]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'url') 
df = df.sort_values(by=['url', 'title'], ascending=[True, True])

df.to_json("consolidated.json", orient="records", force_ascii=False)
df

,auto_format,title,twitter,url,user,tags
0,False,Getting Tensorflow to work with GPU NVidia GTX...,True,http://abhay.harpale.net/blog/machine-learning...,dataradar,
1,False,"Deep Learning - What, Why and Applications",True,http://aiehive.com/deep-learning-applications/,dataradar,
2,False,Normal and Negative Binomial Distributions,True,http://allendowney.blogspot.fr/2016/05/binomia...,dataradar,
3,False,Probability is hard - Part 3,True,http://allendowney.blogspot.fr/2016/05/probabi...,dataradar,
4,False,Probability is hard - Part 2,True,http://allendowney.blogspot.fr/2016/05/probabi...,dataradar,
5,False,Probability is hard - Part 1,True,http://allendowney.blogspot.fr/2016/05/probabi...,dataradar,
6,False,What is a distribution?,True,http://allendowney.blogspot.fr/2016/06/what-is...,dataradar,
7,False,Twitter sentiment analysis with R,True,http://analyzecore.com/2014/04/28/twitter-sent...,dataradar,
8,False,Twitter sentiment analysis based on affective ...,True,http://analyzecore.com/2014/05/11/twitter-sent...,dataradar,
9,False,Cohort analysis with R - Layer-cake graph - Pa...,True,http://analyzecore.com/2014/05/31/cohort-analy...,dataradar,


## Hashtag Automation

In [69]:
max_hashtags = 5

def hashtags_free_number(df, index):
    tags = df.iloc[index]['tags']
        
    str_list = tags.split(",")
    str_list = filter(None, str_list) # fastest
    
    return max_hashtags - len(str_list)

In [70]:
def check_substr(input_str, keyword):
    tmp_str = str.lower(input_str)
    tmp_str = str.replace(tmp_str, ' ', '')
    tmp_str = str.replace(tmp_str, '-', '')
    tmp_str = str.replace(tmp_str, '_', '')
    tmp_str = str.replace(tmp_str, '/', '')
    tmp_str = str.replace(tmp_str, '#', '')
    tmp_str = str.replace(tmp_str, '?', '')
    
    tmp_keyword = str.lower(keyword)
    tmp_keyword = str.replace(tmp_keyword, ' ', '')
    tmp_keyword = str.replace(tmp_keyword, '-', '')
    tmp_keyword = str.replace(tmp_keyword, '_', '')
    tmp_keyword = str.replace(tmp_keyword, '/', '')
    tmp_keyword = str.replace(tmp_keyword, '#', '')
    tmp_keyword = str.replace(tmp_keyword, '?', '')
    
    if tmp_keyword in tmp_str:
        return True
    else:
        return False

In [71]:
def check_pattern(df, index, pattern):
    
    if check_substr(df.iloc[index]['title'].encode("utf-8"), pattern):
        return True
    elif check_substr(df.iloc[index]['url'].encode("utf-8"), pattern):
        return True
    else:
        return False

In [101]:
def set_keyword(df, index, keyword, pattern = ""):
     
    if pattern != "" and (not check_pattern(df, index, pattern)):
        return False
        
    tags = df.iloc[index]['tags']
        
    str_list = tags.split(",")
    str_list = filter(None, str_list) # fastest
    
    if keyword not in str_list:
        str_list.append(keyword)
        df.set_value(index, "tags", ",".join(str_list))
        
    return True

In [102]:
##################### LOADING JSON ##################################
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'url') 
df = df.sort_values(by=['url', 'title'], ascending=[True, True])

###################### PARSING JSON #################################
posts_full = 0

keywords = [
            ["Deep Learning", "DeepLearning"],
            ["TensorFlow", "TensorFlow"],
            ["Keras", "Keras"],
            ["Theano", "Theano"],
            ["Caffe", "Caffe"],
            ["MXNet", "MXNet"],
            ["Python", "Python"],
            ["Convol", "CNN"],
            ["Convol", "ConvNet"],
            ["Recur", "RNN"],
            ["Bayes", "Bayes"],
            ["Adversa", "GAN"],
            ["Aggreg", "Aggregation"],
            ["Algorithm", "Algorithm"],
            ["AWS", "Bayes"],
            ["Analytic", "Analytic"],
            ["Hadoop", "Hadoop"],
            ["Spark", "Spark"],
            ["Flink", "Flink"],
            ["Spark", "Apache"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
    
            
            ["ABTest", "ABTest"],
            ["Accuracy", "Accuracy"],
            ["AddOn", "AddOn"],
            ["AWS", "AWS"],
            ["GCE", "GCE"],
            ["AWS", "Cloud"],
            ["Angular", "AngularJS"],
            ["API", "API"],
            ["Rest", "REST"],
            ["PostgreSQL", "PostgreSQL"],
            ["MySQL", "MySQL"],
            ["Arxiv", "Arxiv"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["Bayes", "Bayes"],
            ["3D", "3D"],
            
            
            
            ["Deep Learning", "NeuralNet"],
        ]

for index in range(df.shape[0]):
    max_iteration = hashtags_free_number(df, index)
    
    if max_iteration > 0:
        
        for keyword in keywords:
            if set_keyword(df, index, keyword[1], keyword[0]):
                max_iteration -= 1
            
                if max_iteration <= 0:
                    posts_full += 1
                    break
                
        keyword_completing = [
            "Data",
            "DataScience",
            "AI"
        ]
        
        if max_iteration > 0 and df.iloc[index]['user'] == "dataradar":
            
            for keyword in keyword_completing:
                if set_keyword(df, index, keyword):
                    max_iteration -= 1
            
                    if max_iteration <= 0:
                        posts_full += 1
                        break
        
    else:
        posts_full += 1
        
print "Posts Full = " + str(posts_full)+"/"+str(df.shape[0])

Posts Full = 80/518


In [103]:
#df.to_json("consolidated.json", orient="records", force_ascii=False)
df

,auto_format,tags,title,twitter,url,user
0,False,"DeepLearning,TensorFlow,NeuralNet,Data,DataSci...",Getting Tensorflow to work with GPU NVidia GTX...,True,http://abhay.harpale.net/blog/machine-learning...,dataradar
1,False,"DeepLearning,NeuralNet,Data,DataScience,AI","Deep Learning - What, Why and Applications",True,http://aiehive.com/deep-learning-applications/,dataradar
2,False,"Data,DataScience,AI",Normal and Negative Binomial Distributions,True,http://allendowney.blogspot.fr/2016/05/binomia...,dataradar
3,False,"Data,DataScience,AI",Probability is hard - Part 3,True,http://allendowney.blogspot.fr/2016/05/probabi...,dataradar
4,False,"Data,DataScience,AI",Probability is hard - Part 2,True,http://allendowney.blogspot.fr/2016/05/probabi...,dataradar
5,False,"Data,DataScience,AI",Probability is hard - Part 1,True,http://allendowney.blogspot.fr/2016/05/probabi...,dataradar
6,False,"Data,DataScience,AI",What is a distribution?,True,http://allendowney.blogspot.fr/2016/06/what-is...,dataradar
7,False,"Data,DataScience,AI",Twitter sentiment analysis with R,True,http://analyzecore.com/2014/04/28/twitter-sent...,dataradar
8,False,"Data,DataScience,AI",Twitter sentiment analysis based on affective ...,True,http://analyzecore.com/2014/05/11/twitter-sent...,dataradar
9,False,"Data,DataScience,AI",Cohort analysis with R - Layer-cake graph - Pa...,True,http://analyzecore.com/2014/05/31/cohort-analy...,dataradar


In [108]:
data = []

for i in range (100):
    data.append(0)
    
for index in range(df.shape[0]):
    tags = df.iloc[index]['tags']
        
    str_list = tags.split(",")
    str_list = filter(None, str_list) # fastest
    
    data[len(str_list)] += 1
    
for i in range (100):
    if data[i] != 0:
        print "Number of rows with " + str(i) + " Hashtags : " + str(data[i])

Number of rows with 0 Hashtags : 98
Number of rows with 1 Hashtags : 32
Number of rows with 2 Hashtags : 4
Number of rows with 3 Hashtags : 225
Number of rows with 4 Hashtags : 87
Number of rows with 5 Hashtags : 72
